In [2]:
import pandas as pd
import random

def extract_score_rank(string):
    info = list(string.split())
    info[1] = info[1][1:-1]
    return float(info[0]),float(info[1])

def extract_name_town_city(sample):
    no_name, town, city = '', '', ''
    res = sample.split(',')

    if len(res) > 0: no_name = res[0]
    if len(res) > 1: town = res[1]
    if len(res) > 2: city = res[2]

    res = no_name.split(' - ') 
    number, name = '', ''
    if len(res) > 0: number = res[0]
    if len(res) > 1: name = res[1]
    
    return number, name, town, city

def extraction(file_names: list, extenstions: list):
    res = []
    for index, file_name in enumerate(file_names):
        table = pd.read_csv(file_name)

        for i in range(table.shape[0]):
            if str(table['Score'][i]) == "nan":
                table['ClgName'][i-1] = str(table['ClgName'][i-1])
                table['ClgName'][i-1] += ", " + str(table['ClgName'][i])
        table = table.replace(r'.*CET', 'MHTCET', regex=True)
        table = table.replace(r'\n', '', regex=True)


        table = table.drop(columns=['ChoiceCode'])
        table = table.dropna(subset=['Score'])
        
        rank = 'Rank' + extensions[index]
        scores = 'Score' + extensions[index]
        table[rank], table[scores] = zip(*table['Score'].map(extract_score_rank))
        table['ClgID'], table['Name'], table['Town'], table['City'] = zip(*table['ClgName'].map(extract_name_town_city))
                
        table = table.drop(columns=['Score', 'ClgName'])
        table = table.reindex(columns=['ClgID', 'Name', 'Branch', 'Town', 'City', 'Exam', rank, scores])
        res.append(table)
    return res

In [11]:
file_names = ['jee19-20cap1.csv', 'jee19-20cap2.csv', 'jee20-21cap1.csv', 'jee20-21cap2.csv']
extensions = ['_19_R1', '_19_R2', '_20_R1', '_20_R2']
dataframes = extraction(file_names, extensions)

In [ ]:
dataframes[0].head()

In [3]:
merge_on = ['ClgID', 'Branch', 'Exam']
res = dataframes[0]
for dataframe in dataframes[1:]:
    res = pd.merge(res, dataframe, left_on=merge_on, right_on=merge_on)
    for col in res.columns:
        if '_' in col:
            name_type = col.split('_')
            if len(name_type) > 2: continue
            colname, t = name_type
            if t == 'x': res[colname] = res[col]
            res.drop(col, axis='columns', inplace=True)


res.head()

,ClgID,Branch,Exam,Rank_19_R1,Score_19_R1,Rank_19_R2,Score_19_R2,Rank_20_R1,Score_20_R1,Rank_20_R2,Score_20_R2,Name,Town,City
0,3215,Computer Engineering,JEE,135.0,210.0,246.0,193.0,56.0,99.141446,82.0,98.892342,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Andheri,Mumbai
1,3036,Chemical Engineering,JEE,256.0,192.0,391.0,179.0,637.0,96.854491,764.0,96.520155,Institute of Chemical Technology,Matunga,Mumbai
2,6271,Computer Engineering,JEE,294.0,188.0,432.0,175.0,178.0,98.403871,215.0,98.257849,Pune Institute of Computer Technology,Dhankavdi,Pune
3,6271,Computer Engineering,JEE,294.0,188.0,432.0,175.0,178.0,98.403871,265.0,98.069304,Pune Institute of Computer Technology,Dhankavdi,Pune
4,6271,Computer Engineering,JEE,294.0,188.0,432.0,175.0,202.0,98.301492,215.0,98.257849,Pune Institute of Computer Technology,Dhankavdi,Pune


In [ ]:
res.shape

In [ ]:
res.head()

In [4]:
colnames = [ 'Name', 'Branch', 'Town', 'City', 'Exam', 'Rank_19_R1', 'Score_19_R1',
       'Rank_19_R2', 'Score_19_R2', 'Rank_20_R1', 'Score_20_R1', 'Rank_20_R2',
       'Score_20_R2']
res = res.reindex(columns=colnames)

In [ ]:
res.to_csv('final.csv')

In [ ]:
res.head()

In [8]:
df = pd.read_csv('final.csv')
moreinfo = pd.read_csv('moreclgdata.csv')
moreinfo.drop(moreinfo.iloc[:, 15:],inplace=True, axis=1)
df = pd.merge(df, moreinfo, left_on='Name', right_on='Name', how='outer')
df = df.drop(columns=['Courses', 'City_x', 'Unnamed: 0'])

/home/phoenix/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
df.head()

,Name,Branch,Town,Exam,Rank_19_R1,Score_19_R1,Rank_19_R2,Score_19_R2,Rank_20_R1,Score_20_R1,...,Total Faculty,Established Year,Rating,University,Facilities,City_y,State,Country,College Type,Average Fees
0,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Computer Engineering,Andheri,JEE,135.0,210.0,246.0,193.0,56.0,99.141446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Information Technology,Andheri,JEE,391.0,179.0,466.0,173.0,116.0,98.688151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Electronics Engineering,Andheri,JEE,1191.0,141.0,1264.0,139.0,1184.0,95.631872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Institute of Chemical Technology,Chemical Engineering,Matunga,JEE,256.0,192.0,391.0,179.0,637.0,96.854491,...,116,2008,3.77,NaN,"Boys Hostel, Girls Hostel, Gym, Library, Sport...",Mumbai,Maharashtra,India,Public/Government,189512.8205
4,Institute of Chemical Technology,Dyestuff Technology,Matunga,JEE,1673.0,128.0,1598.0,130.0,2947.0,92.676883,...,116,2008,3.77,NaN,"Boys Hostel, Girls Hostel, Gym, Library, Sport...",Mumbai,Maharashtra,India,Public/Government,189512.8205


In [10]:
df['Name'].nunique()

4962

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 0 to 228
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Name                       229 non-null    object 
 1   Branch                     229 non-null    object 
 2   Town                       229 non-null    object 
 3   Exam                       229 non-null    object 
 4   Rank_19_R1                 229 non-null    float64
 5   Score_19_R1                229 non-null    float64
 6   Rank_19_R2                 229 non-null    float64
 7   Score_19_R2                229 non-null    float64
 8   Rank_20_R1                 229 non-null    float64
 9   Score_20_R1                229 non-null    float64
 10  Rank_20_R2                 229 non-null    float64
 11  Score_20_R2                229 non-null    float64
 12  Genders Accepted           229 non-null    object 
 13  Campus Size                129 non-null    object 

In [15]:
df['Rating'] = df['Rating'].astype(float)
for i in range(df.shape[0]):
    rank = df['Rank_20_R1'][i]
    if rank < 700: df['Rating'][i] = random.uniform(3.5, 4)
    elif rank < 1100: df['Rating'][i] = random.uniform(3, 3.5)
    elif rank < 2000: df['Rating'][i] = random.uniform(2.5, 3)
    elif rank < 5000: df['Rating'][i] = random.uniform(2, 2.5)
    else: df['Rating'][i] = random.uniform(1, 2)
    
df = df.round({'Rating':2})

<ipython-input-15-36c643b9952d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if rank < 700: df['Rating'][i] = random.uniform(3.5, 4)
<ipython-input-15-36c643b9952d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif rank < 5000: df['Rating'][i] = random.uniform(2, 2.5)
<ipython-input-15-36c643b9952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif rank < 1100: df['Rating'][i] = random.uniform(3, 3.5)
<ipython-input-15-36c6

In [11]:
df.head()

,Name,Branch,Town,Exam,Rank_19_R1,Score_19_R1,Rank_19_R2,Score_19_R2,Rank_20_R1,Score_20_R1,...,Total Faculty,Established Year,Rating,University,Facilities,City_y,State,Country,College Type,Average Fees
0,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Computer Engineering,Andheri,JEE,135.0,210.0,246.0,193.0,56.0,99.141446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Information Technology,Andheri,JEE,391.0,179.0,466.0,173.0,116.0,98.688151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Electronics Engineering,Andheri,JEE,1191.0,141.0,1264.0,139.0,1184.0,95.631872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Institute of Chemical Technology,Chemical Engineering,Matunga,JEE,256.0,192.0,391.0,179.0,637.0,96.854491,...,116,2008,3.77,NaN,"Boys Hostel, Girls Hostel, Gym, Library, Sport...",Mumbai,Maharashtra,India,Public/Government,189512.8205
4,Institute of Chemical Technology,Dyestuff Technology,Matunga,JEE,1673.0,128.0,1598.0,130.0,2947.0,92.676883,...,116,2008,3.77,NaN,"Boys Hostel, Girls Hostel, Gym, Library, Sport...",Mumbai,Maharashtra,India,Public/Government,189512.8205


In [18]:
df.to_csv('database.csv')

In [6]:
df = pd.read_csv('final.csv')
df.head()

,Unnamed: 0,Name,Branch,Town,City,Exam,Rank_19_R1,Score_19_R1,Rank_19_R2,Score_19_R2,Rank_20_R1,Score_20_R1,Rank_20_R2,Score_20_R2
0,0,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Computer Engineering,Andheri,Mumbai,JEE,135.0,210.0,246.0,193.0,56.0,99.141446,82.0,98.892342
1,1,Institute of Chemical Technology,Chemical Engineering,Matunga,Mumbai,JEE,256.0,192.0,391.0,179.0,637.0,96.854491,764.0,96.520155
2,2,Pune Institute of Computer Technology,Computer Engineering,Dhankavdi,Pune,JEE,294.0,188.0,432.0,175.0,178.0,98.403871,215.0,98.257849
3,3,Pune Institute of Computer Technology,Computer Engineering,Dhankavdi,Pune,JEE,294.0,188.0,432.0,175.0,178.0,98.403871,265.0,98.069304
4,4,Pune Institute of Computer Technology,Computer Engineering,Dhankavdi,Pune,JEE,294.0,188.0,432.0,175.0,202.0,98.301492,215.0,98.257849


In [7]:
df['Name'].nunique()

170